# CERR I/O in Slicer 

### Pull imaging data from Slicer viewer and import to CERR for analysis

Setup and module load for Octave oct2py interface

In [ ]:
import os
import numpy as np

In [ ]:
import JupyterNotebooksLib as slicernb

In [ ]:
octave_cli_path = '/usr/local/bin/octave-cli'
os.environ['OCTAVE_EXECUTABLE'] = octave_cli_path

In [ ]:
from oct2py import octave
from oct2py import Struct

In [ ]:
%load_ext oct2py.ipython

In [ ]:
#import octave add-on packages
octave.eval('pkg load image')
octave.eval('pkg load io')
octave.eval('pkg load statistics')
octave.eval('pkg load pythonic')

In [ ]:
octave.warning('off','all');

In [ ]:
javaPath = r'/Library/Java/JavaVirtualMachines/jdk-16.0.2.jdk/Contents/Home/lib/server'
octave.setenv('JAVA_HOME',javaPath)

In [ ]:
cerrPath = r'/Users/elocastro/Dev/CERR'

octave.push('cerrPath',cerrPath)

octave.eval('addpath(cerrPath)')

octave.eval('addToPath2(cerrPath)')

In [ ]:
planCFileName = r'/Users/elocastro/Dev/sliCERR/data/0617-580557_09-09-2000-11778.mat';
octave.push('planCFileName',planCFileName)

In [ ]:
octave.eval('planC = loadPlanC(planCFileName);')

In [ ]:
octave.eval('indexS = planC{end};')

## Import scans to Slicer from CERR

In [ ]:
octave.eval('scanCount = numel(planC{indexS.scan});')
octave.eval('structCount = numel(planC{indexS.structures});')
octave.eval('doseCount = numel(planC{indexS.dose});')

countList = octave.pull(['scanCount','structCount','doseCount'])

#scanCount = octave.pull('scanCount')
#structCount = octave.pull('structCount')
#doseCount = octave.pull('doseCount')

In [ ]:
scanCount   = countList[0]
structCount = countList[1]
doseCount   = countList[2]

In [ ]:
if structCount != 0:
    importStruct = True
else:
    importStruct = False

if doseCount != 0:
    importDose = True
else:
    importDose = False

In [ ]:
scanList = []
structList = []
doseList = []

In [ ]:
if scanCount != 0:
    for i in range(int(scanCount)):
        print('Importing scan' + str(i+1))
        octave.push('i',i+1)
        octave.eval('scanType = planC{indexS.scan}(i).scanType;')
        scanType = octave.pull('scanType')
        octave.eval('structC = {};')
        octave.eval('doseNumV = [];')
        if importStruct:
            print('Import struct option found')
            structNodeName = scanType + '_struct'
            structureNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", structNodeName)            
            for j in range(int(structCount)):
                octave.push('j',j+1)
                octave.eval('if planC{indexS.structures}(j).associatedScan == i; structC{end+1} = planC{indexS.structures}(j).structureName; end')
            octave.eval('assocStructCount = numel(structC);')
        if importDose:
            for k in range(int(doseCount)):
                octave.push('k',k+1)
                #octave.eval('if planC{indexS.dose}(k).associatedScan == i; doseNumV(end+1) = k; end')
                octave.eval('doseNumV = 1:k')
            octave.eval('disp(doseNumV);')
        octave.eval('[affineMat,scan3M_RAS,voxel_size,mask3MC, dose3MC] = getPlanCAffineMat(planC, i, 1, structC, doseNumV);')
        octave.eval("qOffset = affineMat(1:3,end)';")
        octave.eval('scaleMat = eye(3); scaleMat(1,1) = 1/voxel_size(1); scaleMat(2,2) = 1/voxel_size(2); scaleMat(3,3) = 1/voxel_size(3);ijkMat = scaleMat * affineMat(1:3,1:3);')
        affineMat, ijkMat, qOffset, scan3M_RAS, voxel_size = octave.pull(['affineMat','ijkMat','qOffset','scan3M_RAS','voxel_size'])
        if scanType == '':
            scanType = 'planC_scan_' + str(i+1)
        slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", scanType)
        scanList.append(slicer.util.getNode(scanType))
        #volumeNode = slicer.util.getNode(scanType)
        scanList[-1].SetOrigin(qOffset[0])
        #volumeNode.SetOrigin(qOffset[0])
        scanList[-1].SetSpacing(voxel_size[0])
        #volumeNode.SetSpacing(voxel_size[0])
        scanList[-1].SetIJKToRASDirections(ijkMat)
        #volumeNode.SetIJKToRASDirections(ijkMat)
        slicer.util.updateVolumeFromArray(scanList[-1], np.swapaxes(scan3M_RAS,2,0))
        slicer.util.setSliceViewerLayers(background=scanList[-1])
        if importDose:
            print('importing dose')
            octave.eval('dose3M = dose3MC{1};')
            dose3M = octave.pull('dose3M')
            slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", 'dose')
            doseNode = slicer.util.getNode('dose')
            doseNode.SetOrigin(qOffset[0])
            doseNode.SetSpacing(voxel_size[0])
            doseNode.SetIJKToRASDirections(ijkMat)
            slicer.util.updateVolumeFromArray(doseNode,np.swapaxes(dose3M,2,0))
            slicer.util.setSliceViewerLayers(foreground=doseNode)
            slicer.util.setSliceViewerLayers(foregroundOpacity=0.4)
        if importStruct:
            structureNode.SetReferenceImageGeometryParameterFromVolumeNode(scanList[-1])
            for j in range(int(structCount)):
                octave.push('j',j+1)
                octave.eval('structName = structC{j}; mask3M = mask3MC{j};')
                structName, mask3M = octave.pull(['structName','mask3M'])
                print('Importing structure #' + str(j+1) + ', ' + structName)
                slicer.util.addVolumeFromArray(np.swapaxes(mask3M,2,0)*(j+1), ijkToRAS=affineMat, name=structName, nodeClassName="vtkMRMLLabelMapVolumeNode")
                lmap = slicer.util.getNode(structName)
                slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(lmap, structureNode)
                slicer.mrmlScene.RemoveNode(lmap)
                segId = structureNode.GetSegmentation().GetNthSegment(j)
                segId.SetName(structName)
                
                #slicer.util.ImportLabelmapToSegmentationNode(
                #slicer.util.updateSegmentBinaryLabelmapFromArray(mask3M,structureNode,structName,scanList[-1])
            

                

Set viewer with CT base & dose overlay

In [ ]:
for color in ['Red', 'Yellow', 'Green']:
    slicer.app.layoutManager().sliceWidget(color).sliceLogic().GetSliceCompositeNode().SetForegroundVolumeID(doseNode.GetID())
    slicer.app.layoutManager().sliceWidget(color).sliceLogic().GetSliceCompositeNode().SetBackgroundVolumeID(scanList[-1].GetID())

## Pull scan from mrml to CERR

Pull imaging data from the Slicer MRML scene and import to CERR

In [ ]:
#Get preloaded DICOM volume node from MRML scene
volumeNode = scanList[0]

In [ ]:
#Get 3x3 voxel matrix from volume node
imgVolume = np.copy(slicer.util.arrayFromVolume(volumeNode))

#Origin, image spacing
imgOrigin = volumeNode.GetOrigin()
imgSpacing = volumeNode.GetSpacing()

#Get affine matrix with direction cosines, pixdim, origin
imgMatrixVtk = vtk.vtkMatrix4x4()
volumeNode.GetIJKToRASMatrix(imgMatrixVtk)

imgMatrix = np.eye(4)
imgMatrixVtk.DeepCopy(imgMatrix.ravel(),imgMatrixVtk)

imgData = volumeNode.GetImageData()
dtype = imgData.GetScalarSize()

In [ ]:
octave.push('scanName',volumeNode.GetName())
octave.push('qOffset',np.array(imgOrigin))
octave.push('pixDim',np.array(imgSpacing))
octave.push('imgVolume',imgVolume)

In [ ]:
slicer.util.saveNode(volumeNode,os.path.join(os.getcwd(),scanType.replace(' ','_') + '.nii'))

In [ ]:
octave.eval('nii2cerr("' + scanType.replace(' ','_') + '.nii");')

## Run texture mapping

In [ ]:
octave.eval('indexS = planC{end};')

In [ ]:
scanNum = 1;
structNum = 4; #GTV
octave.push('scanNum',scanNum)
octave.push('structNum',structNum)

In [ ]:
octave.eval("paramS = [];")

In [ ]:
octave.eval("fType = 'HaralickCooccurance';")
octave.eval("paramS.NumLevels.val = 64;")
octave.eval("paramS.PatchSize.val = [2 2 0];")
octave.eval("paramS.Directionality.val = 2;")
octave.eval("paramS.Type.val = 'All';")
octave.eval("label = 'textureMap2';")

In [ ]:
octave.eval("planC = createTextureMaps(scanNum,structNum,fType,paramS,label,planC);")

In [ ]:
octave.eval('planC{indexS.texture}')

In [ ]:
octave.eval('scanCount = numel(planC{indexS.scan});')

scanCount = octave.pull('scanCount')
print(scanCount)

### Update viewer

In [ ]:
if scanCount != 0:
    for i in range(int(scanCount)):
        print('Importing scan' + str(i+1))
        octave.push('i',i+1)
        octave.eval('scanType = planC{indexS.scan}(i).scanType;')
        scanType = octave.pull('scanType')
        octave.eval('structC = {};')
        octave.eval('doseNumV = [];')
        if importStruct:
            print('Import struct option found')
            structNodeName = scanType + '_struct'
            structureNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", structNodeName)            
            for j in range(int(structCount)):
                octave.push('j',j+1)
                octave.eval('if planC{indexS.structures}(j).associatedScan == i; structC{end+1} = planC{indexS.structures}(j).structureName; end')
            octave.eval('assocStructCount = numel(structC);')
        if importDose:
            for k in range(int(doseCount)):
                octave.push('k',k+1)
                #octave.eval('if planC{indexS.dose}(k).associatedScan == i; doseNumV(end+1) = k; end')
                octave.eval('doseNumV = 1:k')
            octave.eval('disp(doseNumV);')
        octave.eval('[affineMat,scan3M_RAS,voxel_size,mask3MC, dose3MC] = getPlanCAffineMat(planC, i, 1, structC, doseNumV);')
        octave.eval("qOffset = affineMat(1:3,end)';")
        octave.eval('scaleMat = eye(3); scaleMat(1,1) = 1/voxel_size(1); scaleMat(2,2) = 1/voxel_size(2); scaleMat(3,3) = 1/voxel_size(3);ijkMat = scaleMat * affineMat(1:3,1:3);')
        affineMat, ijkMat, qOffset, scan3M_RAS, voxel_size = octave.pull(['affineMat','ijkMat','qOffset','scan3M_RAS','voxel_size'])
        if scanType == '':
            scanType = 'planC_scan_' + str(i+1)
        slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", scanType)
        scanList.append(slicer.util.getNode(scanType))
        #volumeNode = slicer.util.getNode(scanType)
        scanList[-1].SetOrigin(qOffset[0])
        #volumeNode.SetOrigin(qOffset[0])
        scanList[-1].SetSpacing(voxel_size[0])
        #volumeNode.SetSpacing(voxel_size[0])
        scanList[-1].SetIJKToRASDirections(ijkMat)
        #volumeNode.SetIJKToRASDirections(ijkMat)
        slicer.util.updateVolumeFromArray(scanList[-1], np.swapaxes(scan3M_RAS,2,0))

In [ ]:
100/91.4

In [ ]:
0.188 * 60

## Run Segmentation

In [ ]:
octave.eval('init_ML_DICOM')

In [ ]:
os.getcwd()

In [ ]:
segdir = r'/Users/elocastro/Dev/slicerr'
octave.push('segdir',segdir)

condaEnvName = r'/Users/elocastro/Dev/slicerr/condapack_mac_CT_Heart_DeepLab'
octave.push('condaEnvName',condaEnvName)

algorithm = r'CT_Pericardium_DeepLab'
octave.push('algorithm',algorithm)

In [ ]:
octave.eval('planC = runSegForPlanCInCondaEnv(planC,segdir,algorithm,condaEnvName)')

In [ ]:
octave.save_planC(planC,[],'passed','slicer_0617.mat')

Visualize PERICARDIUM structure in Slicer

In [ ]:
octave.eval('structureListC = {planC{indexS.structures}.structureName};')
strName = 'PERICARDIUM'
octave.push('strName',strName)
octave.eval("structNum = getMatchingIndex(lower(strName),lower(structureListC),'exact');")
octave.eval('mask3M = getStrMask(structNum, planC);')
mask3M = octave.pull('mask3M')

In [ ]:
##set up segmentation nodes
segLabelString = 'PERICARDIUM'
segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", segLabelString)
segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(volumeNode)